In [1]:
from bokeh.plotting import output_file, figure, show
from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import ColumnDataSource

In [2]:
img_source = ColumnDataSource({'image': []})

output_notebook()
image_figure = figure()
image_bokeh = image_figure.image_rgba(image='image', source=img_source,x=0,y=0, dw=10, dh=10)
handle_image = show(image_figure, notebook_handle=True)

Loading BokehJS ...

py_realsense example:
https://github.com/IntelRealSense/librealsense/blob/master/wrappers/python/examples/t265_stereo.py

In [3]:
def prepare_gray_img_for_boke(frame):
    M, N = frame.shape
    img = np.empty((M, N), dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((M, N, 4))
    view[:,:,0] = frame[:,:] # copy red channel
    view[:,:,1] = frame[:,:] # copy blue channel
    view[:,:,2] = frame[:,:] # copy green channel
    view[:,:,3] = 255

    return img[::-1] # flip for Bokeh
#     return img

In [4]:
import pyrealsense2 as rs
import numpy as np

# Declare RealSense pipeline, encapsulating the actual device and sensors
pipe = rs.pipeline()
cfg = rs.config()
pipe.start(cfg)

profiles = pipe.get_active_profile()
streams = {"left"  : profiles.get_stream(rs.stream.fisheye, 1).as_video_stream_profile(),
           "right" : profiles.get_stream(rs.stream.fisheye, 2).as_video_stream_profile()}
intrinsics = {"left"  : streams["left"].get_intrinsics(),
              "right" : streams["right"].get_intrinsics()}

print("Left camera:",  intrinsics["left"])
print("Right camera:", intrinsics["right"])

Left camera: [ 848x800  p[418.394 396.013]  f[286.978 287.115]  Kannala Brandt4 [-0.0084375 0.0427957 -0.0398562 0.00718452 0] ]
Right camera: [ 848x800  p[416.496 401.766]  f[286.996 287]  Kannala Brandt4 [-0.00978763 0.0460905 -0.0429621 0.00803568 0] ]


In [5]:
# try:
for _ in range(50):
    # Wait for the next set of frames from the camera
    frames = pipe.wait_for_frames()
    f1 = frames.get_fisheye_frame(0).as_video_frame()
    f2 = frames.get_fisheye_frame(1).as_video_frame()
    left_data = np.asanyarray(f1.get_data())
    right_data = np.asanyarray(f2.get_data())
    img_source.data = {'image': [prepare_gray_img_for_boke(left_data)]}
    push_notebook(handle=handle_image)

    # Fetch pose frame
    pose = frames.get_pose_frame()
    if pose:
        # Print some of the pose data to the terminal
        data = pose.get_pose_data()
        print("Frame #{}".format(pose.frame_number))
#         print("Position: {}".format(data.translation))
#         print("Velocity: {}".format(data.velocity))
#         print("Acceleration: {}\n".format(data.acceleration))

# finally:
#     pipe.stop()
pipe.stop()

Frame #5
Frame #12
Frame #19
Frame #25
Frame #32
Frame #38
Frame #45
Frame #52
Frame #58
Frame #65
Frame #72
Frame #78
Frame #85
Frame #92
Frame #98
Frame #105
Frame #112
Frame #118
Frame #125
Frame #132
Frame #138
Frame #145
Frame #152
Frame #158
Frame #165
Frame #172
Frame #178
Frame #185
Frame #191
Frame #198
Frame #205
Frame #211
Frame #218
Frame #225
Frame #231
Frame #238
Frame #245
Frame #251
Frame #258
Frame #265
Frame #271
Frame #278
Frame #285
Frame #292
Frame #299
Frame #305
Frame #312
Frame #318
Frame #325
Frame #332
